In [1]:
import pandas as pd
import tweepy
import json
import requests
import os
import ast

# Gathering:

### WeRateDogs Twitter Archive:

In [2]:
archive = pd.read_csv('twitter-archive-enhanced.csv'); archive.head(1)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None


In [3]:
archive.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

### Tweet Image Predictions:

In [4]:
folder_name = 'tweet_image_predictions'

if not os.path.exists(folder_name):
    os.makedirs(folder_name)
    
url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'

response = requests.get(url)

In [5]:
with open(os.path.join(folder_name, url.split('/')[-1]), mode = 'wb') as file:
    file.write(response.content)

In [6]:
image_preds = pd.read_csv('./tweet_image_predictions/image-predictions.tsv', sep = '\t'); image_preds.head()

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True


### Twitter API:

In [7]:
# initialize the credentials of Twitter Developer Account
consumer_key = '###'
consumer_secret = '###'
access_token = '###'
access_secret = '###'

# authenticate as per tweepy docs
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

# create api object
api = tweepy.API(auth, wait_on_rate_limit = True, wait_on_rate_limit_notify = True)

In [8]:
         # example:
tweet = api.get_status(892420643555336193)
tweet._json['favorite_count']

37448

In [9]:
# make folder to store the twitter api data
folder_name = 'twitter_api_data'
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

# init counter to keep track of tweets collected and of the failed ids
i=1
failed_ids = []

# open file to write json objects from api
with open('./twitter_api_data/tweet_json.txt', 'w') as outfile:
    for _ in archive.tweet_id:
        
        # try-except block since few tweet IDs in the archive may have been deleted
        try: 
            tweet = api.get_status(_, tweet_mode = 'extended')
        except:
            failed_ids.append(_)
            i = i+1
            continue
        
        # print the number of tweets collected
        print(f'{_} | {i} of {len(archive.tweet_id)}')
        i = i+1
        
        # dump the json object corresponding to the tweet collected from the api
        json.dump(tweet._json, outfile, indent=4)
        outfile.write('\n')

892420643555336193 | 1 of 2356
892177421306343426 | 2 of 2356
891815181378084864 | 3 of 2356
891689557279858688 | 4 of 2356
891327558926688256 | 5 of 2356
891087950875897856 | 6 of 2356
890971913173991426 | 7 of 2356
890729181411237888 | 8 of 2356
890609185150312448 | 9 of 2356
890240255349198849 | 10 of 2356
890006608113172480 | 11 of 2356
889880896479866881 | 12 of 2356
889665388333682689 | 13 of 2356
889638837579907072 | 14 of 2356
889531135344209921 | 15 of 2356
889278841981685760 | 16 of 2356
888917238123831296 | 17 of 2356
888804989199671297 | 18 of 2356
888554962724278272 | 19 of 2356
888078434458587136 | 21 of 2356
887705289381826560 | 22 of 2356
887517139158093824 | 23 of 2356
887473957103951883 | 24 of 2356
887343217045368832 | 25 of 2356
887101392804085760 | 26 of 2356
886983233522544640 | 27 of 2356
886736880519319552 | 28 of 2356
886680336477933568 | 29 of 2356
886366144734445568 | 30 of 2356
886267009285017600 | 31 of 2356
886258384151887873 | 32 of 2356
88605416005907251

842163532590374912 | 265 of 2356
842115215311396866 | 266 of 2356
841833993020538882 | 267 of 2356
841680585030541313 | 268 of 2356
841439858740625411 | 269 of 2356
841320156043304961 | 270 of 2356
841314665196081154 | 271 of 2356
841077006473256960 | 272 of 2356
840761248237133825 | 273 of 2356
840728873075638272 | 274 of 2356
840698636975636481 | 275 of 2356
840696689258311684 | 276 of 2356
840632337062862849 | 277 of 2356
840370681858686976 | 278 of 2356
840268004936019968 | 279 of 2356
839990271299457024 | 280 of 2356
839549326359670784 | 281 of 2356
839290600511926273 | 282 of 2356
839239871831150596 | 283 of 2356
838952994649550848 | 284 of 2356
838921590096166913 | 285 of 2356
838916489579200512 | 286 of 2356
838831947270979586 | 287 of 2356
838561493054533637 | 288 of 2356
838476387338051585 | 289 of 2356
838201503651401729 | 290 of 2356
838150277551247360 | 291 of 2356
838085839343206401 | 292 of 2356
838083903487373313 | 293 of 2356
837820167694528512 | 294 of 2356
8374822493

810896069567610880 | 518 of 2356
810657578271330305 | 519 of 2356
810284430598270976 | 520 of 2356
810254108431155201 | 521 of 2356
809920764300447744 | 522 of 2356
809808892968534016 | 523 of 2356
809448704142938112 | 524 of 2356
809220051211603969 | 525 of 2356
809084759137812480 | 526 of 2356
808838249661788160 | 527 of 2356
808733504066486276 | 528 of 2356
808501579447930884 | 529 of 2356
808344865868283904 | 530 of 2356
808134635716833280 | 531 of 2356
808106460588765185 | 532 of 2356
808001312164028416 | 533 of 2356
807621403335917568 | 534 of 2356
807106840509214720 | 535 of 2356
807059379405148160 | 536 of 2356
807010152071229440 | 537 of 2356
806629075125202948 | 538 of 2356
806620845233815552 | 539 of 2356
806576416489959424 | 540 of 2356
806542213899489280 | 541 of 2356
806242860592926720 | 542 of 2356
806219024703037440 | 543 of 2356
805958939288408065 | 544 of 2356
805932879469572096 | 545 of 2356
805826884734976000 | 546 of 2356
805823200554876929 | 547 of 2356
8055206356

777641927919427584 | 768 of 2356
777621514455814149 | 769 of 2356
777189768882946048 | 770 of 2356
776819012571455488 | 771 of 2356
776813020089548800 | 772 of 2356
776477788987613185 | 773 of 2356
776249906839351296 | 774 of 2356
776218204058357768 | 775 of 2356
776201521193218049 | 776 of 2356
776113305656188928 | 777 of 2356
776088319444877312 | 778 of 2356
775898661951791106 | 779 of 2356
775842724423557120 | 780 of 2356
775733305207554048 | 781 of 2356
775729183532220416 | 782 of 2356
775364825476165632 | 783 of 2356
775350846108426240 | 784 of 2356
775085132600442880 | 786 of 2356
774757898236878852 | 787 of 2356
774639387460112384 | 788 of 2356
774314403806253056 | 789 of 2356
773985732834758656 | 790 of 2356
773922284943896577 | 791 of 2356
773704687002451968 | 792 of 2356
773670353721753600 | 793 of 2356
773547596996571136 | 794 of 2356
773336787167145985 | 795 of 2356
773308824254029826 | 796 of 2356
773247561583001600 | 797 of 2356
773191612633579521 | 798 of 2356
7728774959

Rate limit reached. Sleeping for: 308


...

684594889858887680 | 1628 of 2356
684588130326986752 | 1629 of 2356
684567543613382656 | 1630 of 2356
684538444857667585 | 1631 of 2356
684481074559381504 | 1632 of 2356
684460069371654144 | 1633 of 2356
684241637099323392 | 1634 of 2356
684225744407494656 | 1635 of 2356
684222868335505415 | 1636 of 2356
684200372118904832 | 1637 of 2356
684195085588783105 | 1638 of 2356
684188786104872960 | 1639 of 2356
684177701129875456 | 1640 of 2356
684147889187209216 | 1641 of 2356
684122891630342144 | 1642 of 2356
684097758874210310 | 1643 of 2356
683857920510050305 | 1644 of 2356
683852578183077888 | 1645 of 2356
683849932751646720 | 1646 of 2356
683834909291606017 | 1647 of 2356
683828599284170753 | 1648 of 2356
683773439333797890 | 1649 of 2356
683742671509258241 | 1650 of 2356
683515932363329536 | 1651 of 2356
683498322573824003 | 1652 of 2356
683481228088049664 | 1653 of 2356
683462770029932544 | 1654 of 2356
683449695444799489 | 1655 of 2356
683391852557561860 | 1656 of 2356
68335797314247

Rate limit reached. Sleeping for: 299


676975532580409345 | 1801 of 2356
676957860086095872 | 1802 of 2356
676949632774234114 | 1803 of 2356
676948236477857792 | 1804 of 2356
676946864479084545 | 1805 of 2356
676942428000112642 | 1806 of 2356
676936541936185344 | 1807 of 2356
676916996760600576 | 1808 of 2356
676897532954456065 | 1809 of 2356
676864501615042560 | 1810 of 2356
676821958043033607 | 1811 of 2356
676819651066732545 | 1812 of 2356
676811746707918848 | 1813 of 2356
676776431406465024 | 1814 of 2356
676617503762681856 | 1815 of 2356
676613908052996102 | 1816 of 2356
676606785097199616 | 1817 of 2356
676603393314578432 | 1818 of 2356
676593408224403456 | 1819 of 2356
676590572941893632 | 1820 of 2356
676588346097852417 | 1821 of 2356
676582956622721024 | 1822 of 2356
676575501977128964 | 1823 of 2356
676533798876651520 | 1824 of 2356
676496375194980353 | 1825 of 2356
676470639084101634 | 1826 of 2356
676440007570247681 | 1827 of 2356
676430933382295552 | 1828 of 2356
676263575653122048 | 1829 of 2356
67623736539290

671542985629241344 | 2042 of 2356
671538301157904385 | 2043 of 2356
671536543010570240 | 2044 of 2356
671533943490011136 | 2045 of 2356
671528761649688577 | 2046 of 2356
671520732782923777 | 2047 of 2356
671518598289059840 | 2048 of 2356
671511350426865664 | 2049 of 2356
671504605491109889 | 2050 of 2356
671497587707535361 | 2051 of 2356
671488513339211776 | 2052 of 2356
671486386088865792 | 2053 of 2356
671485057807351808 | 2054 of 2356
671390180817915904 | 2055 of 2356
671362598324076544 | 2056 of 2356
671357843010908160 | 2057 of 2356
671355857343524864 | 2058 of 2356
671347597085433856 | 2059 of 2356
671186162933985280 | 2060 of 2356
671182547775299584 | 2061 of 2356
671166507850801152 | 2062 of 2356
671163268581498880 | 2063 of 2356
671159727754231808 | 2064 of 2356
671154572044468225 | 2065 of 2356
671151324042559489 | 2066 of 2356
671147085991960577 | 2067 of 2356
671141549288370177 | 2068 of 2356
671138694582165504 | 2069 of 2356
671134062904504320 | 2070 of 2356
67112220491924

667211855547486208 | 2283 of 2356
667200525029539841 | 2284 of 2356
667192066997374976 | 2285 of 2356
667188689915760640 | 2286 of 2356
667182792070062081 | 2287 of 2356
667177989038297088 | 2288 of 2356
667176164155375616 | 2289 of 2356
667174963120574464 | 2290 of 2356
667171260800061440 | 2291 of 2356
667165590075940865 | 2292 of 2356
667160273090932737 | 2293 of 2356
667152164079423490 | 2294 of 2356
667138269671505920 | 2295 of 2356
667119796878725120 | 2296 of 2356
667090893657276420 | 2297 of 2356
667073648344346624 | 2298 of 2356
667070482143944705 | 2299 of 2356
667065535570550784 | 2300 of 2356
667062181243039745 | 2301 of 2356
667044094246576128 | 2302 of 2356
667012601033924608 | 2303 of 2356
666996132027977728 | 2304 of 2356
666983947667116034 | 2305 of 2356
666837028449972224 | 2306 of 2356
666835007768551424 | 2307 of 2356
666826780179869698 | 2308 of 2356
666817836334096384 | 2309 of 2356
666804364988780544 | 2310 of 2356
666786068205871104 | 2311 of 2356
66678179225549

In [14]:
len(failed_ids)

29

> After querying each tweet ID, you will write its JSON data to the required `tweet_json.txt` file with each tweet's JSON data on its own line. You will then read this file, line by line, to create a pandas DataFrame that you will soon assess and clean. This Reading and Writing JSON to a File in Python article from Stack Abuse, will be useful.

In [28]:
#load the json data and store it in a list
data = []
with open('./twitter_api_data/tweet_json.txt') as f:    
        for line in f:         
            data.append(json.loads(line.replace("\'", "\"")))

df_api = pd.DataFrame(data)
#select columns of interest

columns_of_interest = ['id', 'retweet_count', 'favorite_count']
df_api = df_api[columns_of_interest]

JSONDecodeError: Expecting property name enclosed in double quotes: line 2 column 1 (char 2)

In [26]:
df = pd.DataFrame(columns=['tweet_id', 'retweet_count', 'favorite_count'])
with open('./twitter_api_data/tweet_json.txt') as f:
    for line in f:
        status  = json.loads(line)
        tweet_id = status['id_str']
        retweet_count = status['retweet_count']
        favorite_count = status['favorite_count']
        df = df.append(pd.DataFrame([[tweet_id, retweet_count, favorite_count]],
                                        columns=['tweet_id', 'retweet_count', 'favorite_count']))
df = df.reset_index(drop=True)
df

JSONDecodeError: Expecting property name enclosed in double quotes: line 2 column 1 (char 2)